In [1]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
torch.manual_seed(1212)
device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

2024-04-24 22:06:14.915209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 22:06:14.915351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 22:06:15.049502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model = "t5-base"
sep='[SEP]'

In [4]:
# data=pd.read_json(PATH+"train.jsonl",lines=True,encoding='utf-8')
# data.head()

In [5]:
def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    # y = data["spoiler"]
    Y=[]
    # for i in y:
    #     Y.append(" ".join(i))

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords','spoiler','tags']]
    dic1=new_data.to_dict(orient='records')

    dic=[]
    for i in dic1:
        # print(i)
        if('multi' not in i['tags']):
            del i['postText']
            del i['targetParagraphs']
            del i['targetTitle']
            del i['targetDescription']
            del i['targetKeywords']
            del i['tags']
            del i['spoiler']
            del i
            continue

        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        j={}
        j['question']="question : "+i['postText']+"?"
        j['context']="context : "+i['targetParagraphs']
        dic.append(j)
        Y.append(" ".join(i['spoiler']))

        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']
        del i['tags']
        del i['spoiler']
        # break
    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
#     print(dic)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)

    # return X_q[:100],X_c[:100],Y[:100]
    return X_q,X_c,Y



In [6]:
x_train_q,x_train_c,y_train = get_split_data(PATH+'train.jsonl')
x_test_q,x_test_c,y_test = get_split_data(PATH+'test.jsonl',False)
x_val_q,x_val_c,y_val = get_split_data(PATH+'validation.jsonl',False)

In [7]:
x_train_q.shape,x_train_c.shape,len(y_train),x_test_q.shape,x_test_c.shape,len(y_test),x_val_q.shape,x_val_c.shape,len(y_val)

((559,), (559,), 559, (174,), (174,), 174, (143,), (143,), 143)

In [8]:
# create pd dataframe
train_df = pd.DataFrame(list(zip(x_train_q, x_train_c, y_train)), columns =['question', 'context', 'spoiler'])
val_df = pd.DataFrame(list(zip(x_val_q, x_val_c, y_val)), columns =['question', 'context', 'spoiler'])
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


In [9]:
# train_df["question"][0],train_df["context"][0],train_df["spoiler"][0]

In [10]:
train_df.isna().sum(),val_df.isna().sum(),test_df.isna().sum()

(question    0
 context     0
 spoiler     0
 dtype: int64,
 question    0
 context     0
 spoiler     0
 dtype: int64,
 question    0
 context     0
 spoiler     0
 dtype: int64)

In [11]:
# t5
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_data = Dataset(pa.Table.from_pandas(train_df))
val_data = Dataset(pa.Table.from_pandas(val_df))
test_data = Dataset(pa.Table.from_pandas(test_df))
train_data,val_data,test_data

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 559
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 143
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 174
 }))

In [13]:

# train_data[0]

In [14]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        a.append(data['spoiler'][i])

    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs


In [15]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/143 [00:00<?, ? examples/s]

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 559
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 143
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 174
 }))

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [17]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [19]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [20]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)
    
    print(preds)
    for i in preds:
        if(i==""):
            print("empty string")
    for i in labels:
        if(i==""):
            print("empty string")
    
    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))


    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
#     compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240424_220644-ef8xd5k8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fragrant-firebrand-58
wandb: ⭐️ View project at https://wandb.ai/ragha/huggingface
wandb: 🚀 View run at https://wandb.ai/ragha/huggingface/runs/ef8xd5k8


Epoch,Training Loss,Validation Loss
1,No log,1.247030
2,No log,0.804088
3,No log,0.692582
4,No log,0.642103
5,No log,0.621872
6,No log,0.603893
7,No log,0.594705
8,1.315300,0.589599
9,1.315300,0.585784
10,1.315300,0.584273


TrainOutput(global_step=840, training_loss=0.9826794215611049, metrics={'train_runtime': 864.673, 'train_samples_per_second': 7.758, 'train_steps_per_second': 0.971, 'total_flos': 4084889529876480.0, 'train_loss': 0.9826794215611049, 'epoch': 12.0})

In [22]:
trainer.save_model("T5_overall")

In [23]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

{'eval_loss': 0.5293617844581604, 'eval_runtime': 5.3561, 'eval_samples_per_second': 32.486, 'eval_steps_per_second': 4.107, 'epoch': 12.0}


In [25]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

In [26]:
results = trainer.evaluate(eval_dataset=tokenized_train, compute_metrics=compute_metrics)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'eval_loss': 0.4348536431789398, 'eval_runtime': 17.8253, 'eval_samples_per_second': 31.36, 'eval_steps_per_second': 3.927, 'epoch': 12.0}


KeyError: 'eval_bp'

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_val)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [ ]:
q='question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?'
c='context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ " Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy. "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something." Maybe Brady will put a good word in for Welker down in Foxboro if the former Patriots wide receiver impresses him enough.'
enc=tokenizer(q+" "+c,return_tensors='pt',padding=True,truncation=True,max_length=512)
out=model.generate(input_ids=enc['input_ids'].to(device))

In [ ]:
tokenizer.decode(out[0]),'how about that morning we go throw?'

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))